# AgentCore Runtime でのランタイムコンテキストとセッション管理の理解

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime でのランタイムコンテキストとセッション管理の理解と操作方法を学びます。この例では、AgentCore Runtime がセッションを処理し、複数の呼び出しにわたってコンテキストを維持する方法、およびエージェントがコンテキストオブジェクトを通じてランタイム情報にアクセスする方法を示します。

Amazon Bedrock AgentCore Runtime は各ユーザーインタラクションに対して分離されたセッションを提供し、エージェントが複数の呼び出しにわたってコンテキストと状態を維持できるようにしながら、異なるユーザー間の完全なセキュリティ分離を確保します。

### チュートリアル詳細

| 情報             | 詳細                                                                                           |
|:-----------------|:----------------------------------------------------------------------------------------------|
| チュートリアルタイプ | コンテキストとセッション管理                                                                    |
| エージェントタイプ  | シングル                                                                                        |
| エージェントフレームワーク | Strands Agents                                                                            |
| LLM モデル        | Anthropic Claude Haiku 4.5                                                                      |
| チュートリアル構成  | ランタイムコンテキスト、セッション管理、AgentCore Runtime、Strands Agent と Amazon Bedrock モデル |
| チュートリアル分野  | クロスバーティカル                                                                               |
| 例の複雑さ        | 中級                                                                                            |
| 使用 SDK         | Amazon BedrockAgentCore Python SDK と boto3                                                      |

### チュートリアルアーキテクチャ

このチュートリアルでは、Amazon Bedrock AgentCore Runtime がセッションを管理し、エージェントにコンテキストを提供する方法を探ります。以下をデモンストレーションします：

1. **セッション継続性**：同じセッション ID が複数の呼び出しにわたってコンテキストを維持する方法
2. **コンテキストオブジェクト**：エージェントがコンテキストパラメータを通じてランタイム情報にアクセスする方法
3. **セッション分離**：異なるセッション ID が完全に分離された環境を作成する方法
4. **ペイロードの柔軟性**：ペイロードを通じてエージェントにカスタムデータを渡す方法

デモンストレーション目的で、これらのセッション管理機能を示す Strands Agent を使用します。

    
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### チュートリアルの主な機能

* **セッションベースのコンテキスト管理**：AgentCore Runtime がセッション内でコンテキストを維持する方法の理解
* **ランタイムセッションライフサイクル**：セッションの作成、維持、終了について学ぶ
* **コンテキストオブジェクトアクセス**：コンテキストパラメータを通じてセッション ID などのランタイム情報にアクセス
* **セッション分離**：異なるセッションが完全な分離を提供する方法のデモンストレーション
* **ペイロード処理**：カスタムペイロード構造を通じた柔軟なデータ受け渡し
* **クロス呼び出し状態**：同じセッション内で複数の呼び出しにわたってエージェント状態を維持

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker が実行中

## Amazon Bedrock AgentCore Runtime セッションの理解

コードに入る前に、Amazon Bedrock AgentCore Runtime がセッションを管理する方法を理解することが重要です：

### セッション分離とセキュリティ

AgentCore Runtime は専用の microVM を通じて**完全なセッション分離**を提供します：

- **専用リソース**：各セッションは独自の microVM で実行され、CPU、メモリ、ファイルシステムが分離
- **セキュリティ境界**：ユーザーセッション間の完全な分離によりデータ汚染を防止
- **決定論的クリーンアップ**：セッション完了後、microVM は終了し、メモリはサニタイズされる

### セッションライフサイクル

AgentCore Runtime のセッションは特定のライフサイクルに従います：

1. **作成**：一意の `runtimeSessionId` で最初の呼び出し時に新しいセッションが作成される
2. **アクティブ状態**：セッションがリクエストを処理し、コンテキストを維持
3. **アイドル状態**：セッションはコンテキストを保持しながら次の呼び出しを待機
4. **終了**：以下の理由でセッションが終了：
   - 非アクティブ（15分）
   - 最大ライフタイム（8時間）
   - ヘルスチェック失敗

### コンテキスト永続性

セッション内で、AgentCore Runtime は以下を維持します：
- **会話履歴**：以前のインタラクションとレスポンス
- **アプリケーション状態**：実行中に作成された変数とオブジェクト
- **ファイルシステム**：セッション中に作成または変更されたファイル
- **環境変数**：カスタム設定と構成

### セッション管理のベストプラクティス

- **一意のセッション ID**：各ユーザーまたは会話に対して一意のセッション ID を生成
- **コンテキストの再利用**：関連する呼び出しには同じセッション ID を使用してコンテキストを維持
- **セッション境界**：異なるユーザーまたは無関係な会話には異なるセッション ID を使用
- **一時的な性質**：永続的なデータ保存にセッションを頼らない（永続化には AgentCore Memory を使用）

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## AgentCore Runtime へのデプロイ用にエージェントを準備

それでは、セッション管理とコンテキスト処理をデモンストレーションするためにエージェントを AgentCore Runtime にデプロイしましょう。エージェントは以下の方法を示します：

1. **ランタイムコンテキストへのアクセス**：`context` パラメータを使用してセッション情報を取得
2. **カスタムペイロードの処理**：ペイロードを通じて渡される構造化データを処理
3. **セッション状態の維持**：セッション内でのユーザーインタラクションを追跡
4. **セッション境界のデモンストレーション**：異なるセッションが分離されていることを示す

### コンテキストオブジェクトの理解

AgentCore Runtime の `context` オブジェクトは、現在の実行環境に関する貴重な情報を提供します：

- **session_id**：現在のランタイムセッション識別子
- **ランタイムメタデータ**：ランタイム環境に関する情報
- **実行詳細**：現在の呼び出しに関するコンテキスト

### コンテキスト処理を備えた Strands Agent

セッション管理とコンテキスト処理をデモンストレーションする実装を見てみましょう：

In [ ]:
%%writefile strands_claude_context.py
from strands import Agent, tool
from strands_tools import calculator # calculator ツールをインポート
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime

app = BedrockAgentCoreApp()

# カスタムツールを作成 
@tool
def weather():
    """ 天気を取得 """ # ダミー実装
    return "sunny"

@tool
def get_time():
    """ 現在時刻を取得 """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""
    あなたは親切なアシスタントです。簡単な数学計算ができ、
    天気を教えたり、現在時刻を提供したりできます。
    常にユーザーの名前を認識することから始めてください
    """
)

def get_user_name(user_id):
    users = {
        "1": "Maira",
        "2": "Mani",
        "3": "Mark",
        "4": "Ishan",
        "5": "Dhawal"
    }
    return users[user_id]
    
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    """
    コンテキスト処理とセッション管理をデモンストレーションする AgentCore Runtime エントリーポイント。
    
    Args:
        payload: ユーザーデータとリクエスト情報を含む入力ペイロード
        context: セッションと実行情報を含むランタイムコンテキストオブジェクト
    
    Returns:
        str: コンテキスト情報を組み込んだエージェントのレスポンス
    """
    user_input = payload.get("prompt")
    user_id = payload.get("user_id")
    user_name = get_user_name(user_id)
    
    # ランタイムコンテキスト情報にアクセス
    print("=== ランタイムコンテキスト情報 ===")
    print("ユーザー ID:", user_id)
    print("ユーザー名:", user_name)
    print("ユーザー入力:", user_input)
    print("ランタイムセッション ID:", context.session_id)
    print("コンテキストオブジェクトタイプ:", type(context))
    print("=== コンテキスト情報終了 ===")
    
    # コンテキスト情報を含むパーソナライズされたプロンプトを作成
    prompt = f"""私の名前は {user_name} です。これが私のリクエストです: {user_input}
    
    追加コンテキスト: これはセッション {context.session_id} です。
    私の名前を認識し、サポートをお願いします。"""
    
    response = agent(prompt)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## AgentCore Runtime でのセッション管理の理解

上記のコードは、AgentCore Runtime がセッションを管理し、エージェントにコンテキストを提供する方法についていくつかの重要な概念を示しています：

### コンテキストオブジェクトの構造

エントリーポイント関数の `context` パラメータはランタイム情報へのアクセスを提供します：

```python
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    # セッション情報にアクセス
    session_id = context.session_id
    # エージェントロジックでコンテキスト情報を使用
```

### セッション継続性の利点

単一のセッション内で、AgentCore Runtime は以下を提供します：

1. **永続的な環境**：変数と状態が呼び出しにわたって永続化
2. **コンテキスト保存**：エージェントは以前のインタラクションを参照可能
3. **リソース再利用**：初期化されたモデルとツールがロードされたまま維持
4. **パフォーマンスメリット**：後続の呼び出しでのコールドスタート時間が短縮

### セッション分離の保証

AgentCore Runtime はセッション間の完全な分離を確保します：

- **セキュリティ**：各セッションは独自の microVM で分離されたリソースと共に実行
- **プライバシー**：異なるユーザーセッション間でデータ漏洩なし
- **信頼性**：あるセッションの問題が他のセッションに影響しない
- **クリーンアップ**：セッション終了後の完全なメモリサニタイズ

### ペイロードの柔軟性

`payload` パラメータは柔軟なデータ受け渡しを可能にします：

```python
# ペイロード構造の例
payload = {
    "prompt": "ユーザーの質問",
    "user_id": "1",
    "preferences": {...},
    "context_data": {...}
}
```

これにより、ランタイムが提供するセッションコンテキストを維持しながら、クライアントとエージェント間のリッチで構造化された通信が可能になります。

### AgentCore Runtime デプロイメントの設定

次に、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中、アプリケーションコードに基づいて Docker ファイルが生成されます

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_context.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_claude_context"
)

### コンテキスト対応エージェントを AgentCore Runtime に起動

Docker ファイルができたので、コンテキスト対応エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

エージェントは AgentCore Runtime がセッションを管理し、エージェントにコンテキスト情報を提供する方法をデモンストレーションします。

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore Runtime のステータス確認
AgentCore Runtime をデプロイしたので、デプロイメントステータスを確認しましょう

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

## セッション管理とコンテキスト処理のデモンストレーション

それでは、異なるシナリオをテストして AgentCore Runtime の主要なセッション管理機能をデモンストレーションしましょう：

### シナリオ1：セッション継続性
同じセッション ID を複数の呼び出しで使用して、コンテキストがどのように維持されるかを示します。

### シナリオ2：セッション分離
異なるセッション ID を使用して、セッション間の完全な分離をデモンストレーションします。

### シナリオ3：コンテキスト情報アクセス
エージェントがランタイムコンテキスト情報にアクセスする方法を示します。

<div style="text-align:left">
    <img src="images/invoke.png" width="85%"/>
</div>

In [ ]:
import uuid
import json
from IPython.display import Markdown, display

# セッション継続性をデモンストレーションするためのセッション ID を作成
session_id = uuid.uuid4()
print(f"📋 セッション1 開始: {session_id}")
print(f"👤 ユーザー: Maira (ID: 1)")
print(f"❓ 天気に関する最初の質問\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "外の天気はどうですか？",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# 同じセッション ID を使用してセッション継続性をデモンストレーション
print(f"🔄 セッション1 継続: {session_id}")
print(f"👤 同じユーザー: Maira (ID: 1)")
print(f"❓ 数学に関するフォローアップ質問\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "2x5 はいくらですか？",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# 同じセッション ID を継続 - エージェントが以前の計算を覚えていることに注目
print(f"🔄 セッション1 継続: {session_id}")
print(f"👤 同じユーザー: Maira (ID: 1)")
print(f"❓ 以前の回答に基づく質問 - コンテキスト継続性をデモンストレーション\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "それに 34 を足すと？",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# 新しいセッション - セッション分離をデモンストレーション
# コンテキストが失われることを示すために完全に新しいセッション ID を作成
new_session_id = uuid.uuid4()
print(f"🆕 新しいセッション2 開始: {new_session_id}")
print(f"👤 同じユーザー: Maira (ID: 1)")
print(f"❓ 以前の計算を参照しようとする - セッション分離により失敗するはず\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "それに 10 を足すと？",
    "user_id": "1"
}, session_id=str(new_session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# 新しいセッションと新しいユーザー - 完全な分離をデモンストレーション
different_user_session = uuid.uuid4()
print(f"🆕 セッション3 開始: {different_user_session}")
print(f"👤 異なるユーザー: Mani (ID: 2)")
print(f"❓ 最初のユーザーと同じ質問 - ユーザー分離をデモンストレーション\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "天気はどうですか？",
    "user_id": "2"
}, session_id=str(different_user_session))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

## セッション管理結果の理解

上記のデモンストレーションは、AgentCore Runtime のセッション管理のいくつかの主要な側面を示しています：

### 1. セッション継続性（セッション1）
- **最初の呼び出し**：エージェントが天気の質問に応答し、ユーザー名を認識
- **2回目の呼び出し**：エージェントが計算を実行（2×5=10）
- **3回目の呼び出し**：エージェントが以前の結果を参照（「それに 34 を足すと」= 44）

**主な学び**：エージェントは同じセッション内で複数の呼び出しにわたってコンテキストを維持し、以前のインタラクションからの計算結果を覚えていました。

### 2. セッション分離（セッション2）
- **新しいセッション ID**：完全に新しいセッションを作成
- **同じユーザー**：同じユーザー ID を使用したが異なるセッション
- **コンテキスト喪失**：エージェントは以前の計算を参照できない

**主な学び**：同じユーザーでも、新しいセッションは以前のコンテキストにアクセスできない完全に分離された環境を作成します。

### 3. ユーザーとセッションの分離（セッション3）
- **異なるユーザー**：Maira の代わりに Mani
- **新しいセッション**：以前のセッションからの完全な分離
- **新鮮なコンテキスト**：エージェントはクリーンな状態から開始

**主な学び**：各セッションは完全な分離を提供し、異なるユーザーとインタラクション間のプライバシーとセキュリティを確保します。

### 4. コンテキストオブジェクトの使用
すべての呼び出しを通じて、エージェントは：
- `context.session_id` を通じてランタイムコンテキストにアクセス
- カスタムペイロードデータ（`user_id`、`prompt`）を処理
- ログ記録とデバッグ情報を維持

**主な学び**：コンテキストオブジェクトは、エージェントが機能強化とデバッグに使用できる貴重なランタイム情報を提供します。

### デモンストレーションされたセッション管理のベストプラクティス

1. **会話の継続性のために一貫したセッション ID を使用**
2. **異なるユーザーまたは会話には一意のセッション ID を生成**
3. **強化されたエージェント動作のためにコンテキスト情報を活用**
4. **セッション境界を考慮して設計** - セッション間での永続性を前提としない
5. **セッションが変更または期限切れになったときのコンテキスト喪失を適切に処理**

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
import boto3

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# おめでとうございます！

Amazon Bedrock AgentCore Runtime を使用したセッション管理とコンテキスト処理の実装とテストに成功しました！

## 学んだこと：

### セッション管理の基礎
* **セッション継続性**：同じセッション ID が複数の呼び出しにわたってコンテキストを維持する方法
* **セッション分離**：異なるセッション ID が完全に分離された環境を作成する方法
* **コンテキスト保存**：エージェントが状態を維持し、以前のインタラクションを参照する方法
* **セキュリティ境界**：AgentCore Runtime がユーザー間の完全な分離を確保する方法

### ランタイムコンテキスト処理
* **コンテキストオブジェクトアクセス**：`context` パラメータを通じてランタイム情報にアクセスする方法
* **セッション情報**：エージェントロジックでセッション ID を取得・使用する方法
* **ペイロード処理**：カスタムペイロードを通じて渡される構造化データを処理する方法
* **ランタイムメタデータ**：エージェントが実行環境情報にアクセスする方法

### AgentCore Runtime アーキテクチャ
* **MicroVM 分離**：各セッションは独自の分離された microVM で実行
* **リソース管理**：セッションごとの専用 CPU、メモリ、ファイルシステム
* **セキュリティモデル**：セッション終了後の完全なメモリサニタイズ
* **ライフサイクル管理**：セッション状態（アクティブ、アイドル、終了）とタイムアウト

### ベストプラクティスの実装
* **セッション ID の生成**：異なる会話に対して一意の識別子を作成
* **コンテキストの活用**：強化されたエージェント動作のためにランタイムコンテキストを活用
* **状態管理**：一時的な状態と永続的な状態の理解
* **エラー処理**：コンテキスト喪失とセッション境界の適切な処理